## Hardware-accelerated Reversi Board Detection on PYNQ Platform
2024.07.12 Naoki F., AIT. See LICENSE.txt for license information.

まずはハードウェアオーバーレイを書き込んで，必要な IP コアを参照できるようにします．
もし xsa ファイルをアップロードした場合は，Overlay の引数を適切に修正してください．
古いバージョン (<= 2.6) の PYNQ では，MMIO にアクセスするために mem_dict ではなく ip_dict を使う必要があります．

At first, write a hardware overlay and prepare references to IP cores that has to be controlled.
If you are using xsa file, correct the argument of the Overlay constructor.
In an older version of PYNQ (<= 2.6), use ip_dict instead of ip_dict to access to the MMIO.

In [ ]:
from ReversiAccelerator import *
from pynq.lib.video import *
from pynq import MMIO
from pynq import PL

PL.reset()
accel = ReversiAccelerator("reversi_hw.bit")
hdmi_in = accel.pl.hdmi_in
hdmi_out = accel.pl.hdmi_out
vdma = accel.pl.axi_vdma_0
bram = MMIO(base_addr = accel.pl.mem_dict['axi_bram_ctrl_0']['phys_addr'], length = 129600)

HDMI 入力を初期化します．
まれに PC からの接続の認識が遅れて，このセルの実行に失敗することがあります．
PC から外部モニタの存在を認識できているようであれば，もう1度実行してみてください．

Initialize the HDMI input.
This cell may fail when it takes long time for the PC to recognize the monitor.
If it appears that the monitor has been recognized at last, try again.

In [2]:
hdmi_in.start(10)
frm_w = hdmi_in.mode.width
frm_h = hdmi_in.mode.height
if frm_w != 1920 or frm_h != 1080:
    raise ValueError("HDMI Input (%d, %d) is not 1080p" % (frm_w, frm_h))

vdma.readchannel.mode = VideoMode(1920, 1080, 24, 60)
vdma.readchannel.start()

HDMI 出力を初期化します．
このセルの実行が終われば，ディスプレイに入力と同じ映像が表示されるはずです．

Initialize the HDMI output.
After that, the same image as the input will appear on the display.

In [3]:
import numpy as np

hdmi_out.mode = VideoMode(1920, 1080, 24)
hdmi_out.start()
vdma.writechannel.mode = VideoMode(1920, 1080, 24)
vdma.writechannel.start()
vdma.readchannel.tie(vdma.writechannel)

obuf = np.zeros(shape=(270, 480), dtype=np.uint8)

これがメインループです．ユーザにより明示的に停止されるまで，繰り返し実行されます．

This is the main loop, executed repeatedly until explicitly stopped by the user.

In [4]:
import Visualize as vis
import my_board_recognition as mbr
import board_recognition as br

recognizer = br.RealBoardRecognizer()
hint = br.Hint()
hint.mode = br.Mode.PHOTO

for i in range(100):
    frame = vdma.readchannel.readframe()
    crop = np.array(frame[28:1052, 448:1472])
    lines = accel.start(crop)
    line_groups, thetas = mbr.clusterLinesByTheta(lines)
    line_groups = mbr.checkLineGroups(line_groups, thetas)
    line_groups = mbr.findRepresentativeLines(line_groups)
    ret, result = mbr.findVertices(line_groups, 512, 512)
    obuf[:] = 0
    vis.drawHoughLines(obuf, lines, 240, 135, 0xe0, 4, 1)
    if line_groups is not None:
        vis.drawHoughLines(obuf, line_groups[0], 240, 135, 0x9f, 4, 1)
        vis.drawHoughLines(obuf, line_groups[1], 240, 135, 0x9f, 4, 1)        
    vis.drawFrames(obuf, 480, 270, 256, 256, 0x83)
    if ret:
        ret, result = recognizer.detectDisc(crop, hint, result)
    vis.drawresult(obuf, result if ret else None, 4, 158, 13)
    bram.array[:] = obuf.ravel().view('uint32')

映像の受け取りと送出を停止します．
このセルを実行せずに別のハードウェアオーバーレイを読み込んだりすると，
PYNQ を再起動するまで HDMI 入出力が機能しなくなる場合があります．

Stop receiving and sending video.
If you forget to run this cell before reading another hardware overlay etc.,
HDMI input/output may fall into unknown state and may not function until rebooting PYNQ.

In [5]:
vdma.readchannel.stop()
vdma.writechannel.stop()
hdmi_in.stop()
hdmi_out.stop()